In [10]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to c:\users\user\appdata\local\temp\pip-req-build-kbi82kdb
  Resolved https://github.com/huggingface/transformers to commit 5e0ffd91830f2c8a0133518d1724d6f3ff92a0e1
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'


  Running command git clone -q https://github.com/huggingface/transformers 'C:\Users\User\AppData\Local\Temp\pip-req-build-kbi82kdb'


In [11]:
# Importing stock libraries
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [12]:
# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Preparing for TPU usage
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

In [13]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage for finetuning the model and to prepare it for predictions

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [14]:
# Creating the training function. This will be called in the main function. It is run depending on the epoch value.
# The model is put into train mode and then we wnumerate over the training loader and passed to the defined network 
loss_train = []
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
        loss = outputs[0]
        loss_train.append(loss)
        
        print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [15]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
#             if _%100==0:
            print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:

TRAIN_BATCH_SIZE = 3    # input batch size for training 
VALID_BATCH_SIZE = 3    # input batch size for testing
TRAIN_EPOCHS = 1        # number of epochs to train 
VAL_EPOCHS = 1 
LEARNING_RATE = 1e-4    # learning rate 
SEED = 42               # random seed 
MAX_LEN = 512
SUMMARY_LEN = 150 

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(SEED) # pytorch random seed
  np.random.seed(SEED) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # tokenzier for encoding the text
  tokenizer = T5Tokenizer.from_pretrained("t5-base")

  # Importing and Pre-Processing the domain data
  # Selecting the needed columns only. 
  # Adding the summarzie text in front of the text. This is to format the dataset similar to how T5 model was trained for summarization task. 
  df = pd.read_csv('./datasets/news_summary.csv',encoding='latin-1')
  df = df[['text','ctext']]
  df.ctext = 'summarize: ' + df.ctext
  print(df.head())


  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest will be used for validation. 
  train_size = 0.8
  train_dataset=df.sample(frac=train_size, random_state = SEED).reset_index(drop=True)
  val_dataset=df.drop(train_dataset.index).reset_index(drop=True)

  print("FULL Dataset: {}".format(df.shape))
  print("TRAIN Dataset: {}".format(train_dataset.shape))
  print("TEST Dataset: {}".format(val_dataset.shape))


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)
  val_set = CustomDataset(val_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)

  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': TRAIN_BATCH_SIZE,
      'shuffle': True,
      'num_workers': 0,
      }

  val_params = {
      'batch_size': VALID_BATCH_SIZE,
      'shuffle': False,
      'num_workers': 0
      }

  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)



  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained("t5-base")
  model = model.to(device)

  # Defining the optimizer that will be used to tune the weights of the network in the training session. 
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

  # Training loop
  print('Initiating Fine-Tuning for the model on our dataset')

  for epoch in range(TRAIN_EPOCHS):
      train(epoch, tokenizer, model, device, training_loader, optimizer)


  # Validation loop and saving the resulting file with predictions and acutals in a dataframe.
  # Saving the dataframe as predictions.csv
  print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
  for epoch in range(VAL_EPOCHS):
      predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
      final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
      final_df.to_csv('predictions.csv')
      print('Output Files generated for review')



E:\Anaconda3\lib\site-packages\transformers\models\t5\tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


                                                text  \
0  The Administration of Union Territory Daman an...   
1  Malaika Arora slammed an Instagram user who tr...   
2  The Indira Gandhi Institute of Medical Science...   
3  Lashkar-e-Taiba's Kashmir commander Abu Dujana...   
4  Hotels in Maharashtra will train their staff t...   

                                               ctext  
0  summarize: The Daman and Diu administration on...  
1  summarize: From her special numbers to TV?appe...  
2  summarize: The Indira Gandhi Institute of Medi...  
3  summarize: Lashkar-e-Taiba's Kashmir commander...  
4  summarize: Hotels in Mumbai and other Indian c...  
FULL Dataset: (4514, 2)
TRAIN Dataset: (3611, 2)
TEST Dataset: (903, 2)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Initiating Fine-Tuning for the model on our dataset


E:\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2323: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Loss:  7.483099460601807
Epoch: 0, Loss:  4.2015533447265625


In [ ]:
torch.save(model.state_dict(), './model1.pth')